In [133]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from copy import deepcopy

In [134]:
MODE="test"
SIDES=["left", "right"]
NUM_SAMPLES = 100 
CLIP_DURATION = 10 # Duration in seconds
NUM_CLIPS = 10 

In [135]:
def load_data(path):
    with open(path, "rb") as f:
        data = pd.read_pickle(f)
    return data

def save_data(data, path):
    with open(path, "wb") as f:
        pd.to_pickle(data, f)

In [136]:
# Function to process each row of the DataFrame
def data_augmentation(data:pd.DataFrame, num_samples=100, duration=10, num_clips = 10):
    new=[]
    side="left"
    for idx in range(len(data)):
        row = deepcopy(data.iloc[idx])
        tot_time = row[f"myo_{side}_timestamps"][-1] - row[f"myo_{side}_timestamps"][0]
        duration = min(duration, tot_time)
        highest_offset=max(0, tot_time-duration)
        offsets = np.linspace(0, highest_offset, num_clips)+row[f"myo_{side}_timestamps"][0]
        timestamps = row[f"myo_{side}_timestamps"]
        readings = row[f"myo_{side}_readings"]
        interpolator = interp1d(timestamps, readings, kind='linear', fill_value='extrapolate', axis=0)
        for st in offsets:
            row = deepcopy(data.iloc[idx])
            interpolation_time = np.linspace(st, st + duration, num_samples)  # Assuming the last timestamp represents the end time
            interpolated_data = interpolator(interpolation_time)
            row[f"myo_{side}_timestamps"] = interpolation_time
            row[f"myo_{side}_readings"] = interpolated_data
            new.append(row)
    new = pd.DataFrame(new)
    side="right"
    for idx in range(len(new)):
        row = new.iloc[idx]
        tot_time = row[f"myo_{side}_timestamps"][-1] - row[f"myo_{side}_timestamps"][0]
        duration = min(duration, tot_time)
        highest_offset=max(0, tot_time-duration)
        offsets = np.linspace(0, highest_offset, num_clips)+row[f"myo_{side}_timestamps"][0]
        timestamps = row[f"myo_{side}_timestamps"]
        readings = row[f"myo_{side}_readings"]
        interpolator = interp1d(timestamps, readings, kind='linear', fill_value='extrapolate', axis=0)
        for st in offsets:
            interpolation_time = np.linspace(st, st + duration, num_samples)  # Assuming the last timestamp represents the end time
            interpolated_data = interpolator(interpolation_time)
            new[f"myo_{side}_timestamps"].iloc[idx] = interpolation_time
            new[f"myo_{side}_readings"].iloc[idx] = interpolated_data
    return new

In [137]:
data = load_data(f"data/ActionNet-EMG/ActionNet_{MODE}_emg_processed.pkl")
new_data = data_augmentation(data, num_samples=NUM_SAMPLES, duration=CLIP_DURATION, num_clips=NUM_CLIPS)
print(data["myo_left_readings"].iloc[0].shape)
print("ORiginal data shape", data.shape)
print(new_data["myo_left_readings"].iloc[0].shape)
print("New data shape", new_data.shape)
print(new_data.head())

/var/folders/vv/68yh64d172jf_vsrj05s2kwc0000gn/T/ipykernel_13122/1480370074.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new[f"myo_{side}_timestamps"].iloc[idx] = interpolation_time
/var/folders/vv/68yh64d172jf_vsrj05s2kwc0000gn/T/ipykernel_13122/1480370074.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new[f"myo_{side}_readings"].iloc[idx] = interpolated_data


(7919, 8)
ORiginal data shape (59, 8)
(100, 8)
New data shape (590, 8)
       description         start          stop  \
4  Peel a cucumber  1.655172e+09  1.655172e+09   
4  Peel a cucumber  1.655172e+09  1.655172e+09   
4  Peel a cucumber  1.655172e+09  1.655172e+09   
4  Peel a cucumber  1.655172e+09  1.655172e+09   
4  Peel a cucumber  1.655172e+09  1.655172e+09   

                                 myo_left_timestamps  \
4  [1655172364.999943, 1655172365.100953, 1655172...   
4  [1655172369.247236, 1655172369.348246, 1655172...   
4  [1655172373.494529, 1655172373.595539, 1655172...   
4  [1655172377.741822, 1655172377.842832, 1655172...   
4  [1655172381.989115, 1655172382.090125, 1655172...   

                                   myo_left_readings  \
4  [[-0.9733333333333334, 0.19999999999999996, -0...   
4  [[-0.8133333333333334, -0.6302896911700739, -0...   
4  [[-0.7827855254762556, -0.41333333333333333, -...   
4  [[-0.7333333333333334, -0.38316525806623086, -...   
4  [[-0.52,

In [138]:
save_data(new_data, f"data/ActionNet-EMG/ActionNet_{MODE}_emg_augmented.pkl")